# INVARI: Sensor Fusion Comparison

This notebook compares complementary filter and Extended Kalman Filter performance
on IMU data, computing RMSE metrics and visualizing results.


In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.insert(0, str(Path().parent))

from src.filters import ComplementaryFilter, ExtendedKalmanFilter
from src.io import IMULoader
from src.utils import compute_rmse, compute_orientation_error, plot_comparison


## Load IMU Data


In [ ]:
# Load sample IMU data
data_path = Path("../data/sample_imu.csv")
data = IMULoader.load_csv(data_path)

print(f"Loaded {len(data)} samples")
print(f"Sample rate: {data.sample_rate:.1f} Hz")
print(f"Duration: {data.get_duration():.2f} seconds")


## Process with Complementary Filter


In [ ]:
dt = 1.0 / data.sample_rate
comp_filter = ComplementaryFilter(alpha=0.96, dt=dt)

comp_orientations, comp_quaternions = comp_filter.process_batch(
    data.accelerometer,
    data.gyroscope,
    data.timestamps
)

print("Complementary filter processing complete")


## Process with Extended Kalman Filter


In [ ]:
ekf_filter = ExtendedKalmanFilter(dt=dt)

ekf_orientations, ekf_quaternions, ekf_covariances = ekf_filter.process_batch(
    data.accelerometer,
    data.gyroscope,
    data.timestamps
)

print("EKF processing complete")


## Compute RMSE Metrics

**Note:** Without ground truth, we compare filters against each other.
In practice, you would compare against motion capture or known trajectories.


In [ ]:
# Compare filter outputs
rmse_roll = compute_rmse(comp_orientations[:, 0], ekf_orientations[:, 0])
rmse_pitch = compute_rmse(comp_orientations[:, 1], ekf_orientations[:, 1])
rmse_yaw = compute_rmse(comp_orientations[:, 2], ekf_orientations[:, 2])

# Quaternion error
quat_error = compute_orientation_error(comp_quaternions, ekf_quaternions)
mean_quat_error = np.mean(quat_error)

print("=== RMSE Metrics ===")
print(f"Roll RMSE: {np.degrees(rmse_roll):.3f} deg")
print(f"Pitch RMSE: {np.degrees(rmse_pitch):.3f} deg")
print(f"Yaw RMSE: {np.degrees(rmse_yaw):.3f} deg")
print(f"Mean quaternion error: {np.degrees(mean_quat_error):.3f} deg")


## Visualize Comparison


In [ ]:
plot_comparison(
    data.timestamps,
    None,  # No ground truth available
    comp_orientations,
    ekf_orientations,
    title="Filter Comparison"
)
